### Ideal RSA Model -- Fully Observed Environment and Utterance Space

**What’s in this notebook?** This notebook describes the listener model in the open world Boxworld domain, encoding what pragmatic listener might reasonably have in mind when trying to interpret a given utterance.
This notebook is a prototype that can ultimately be abstracted away for later use.

In [128]:
using Gen
using JSON

include("dirichlet.jl")

**1: A generative model of a stockroom.** 

We'll begin by defining some primitives for the Stockroom environment. These model the same primitives in the boxworld/world.py library used to generate the Stockroom dataset:
a `Box`, an `Aisle` that contains a set of boxes, and a `World` containing a set of aisles.

We'll also define some utilities for serializing and deserializing these data structures.

In [244]:
module Stockroom
MIN_BOX_SIZE = 0.1
MIN_BOX_SPACING = 0.1

    function clamp_min_max(x, min, max)
        if x <= min
            return min
        elseif x >= max
            return max
        else 
            return x
        end
     end 


    mutable struct Box
        id::Int
        pos::Tuple{Int, Float64}  # Pos is in (shelf_id, x_distance on shelf)
        width::Float64
        height::Float64
    end 
    function init_clamped_box(id, pos, width, height, aisle)
        shelf_height = aisle.size / aisle.num_shelves
        width = clamp_min_max(width, MIN_BOX_SIZE, aisle.size - MIN_BOX_SIZE)
        height = clamp_min_max(height, MIN_BOX_SIZE, shelf_height - MIN_BOX_SIZE)
        return Box(id, pos, width, height)
    end
    function deserialize_box(box_dict)
        return Box(box_dict["id"],
                  (box_dict["pos"][1], box_dict["pos"][2]),
                   box_dict["width"],
                   box_dict["height"])
    end 

    mutable struct Aisle
        id::Int
        size::Float64
        num_shelves::Float64
        boxes::Vector{Box}
        function Aisle(id, size, num_shelves, boxes)
            shelf_height = size / num_shelves
            function clamp_boxes!(shelf_height::Float64, boxes::Vector{Box})
                for box in boxes
                    box.width = clamp_min_max(box.width, MIN_BOX_SIZE, size - MIN_BOX_SIZE)
                    box.height = clamp_min_max(box.height, MIN_BOX_SIZE, shelf_height - MIN_BOX_SIZE)
                end 
                return boxes
            end 
            new(id, size, num_shelves, clamp_boxes!(shelf_height, boxes))
        end 
    end 
    Aisle(id, size, num_shelves) = Aisle(id, size, num_shelves, Box[])
    function room_left(aisle::Aisle, box::Box, distance::Float64)
        # Returns true if it is possible to place the box on the current aisle.
        # Assumes boxes are sorted.
        if length(aisle.boxes) < 1
            last_box_shelf = 1
            last_box_start = 0
            last_box_width = 0
        else
            last_box = last(aisle.boxes)
            last_box_shelf = last_box.pos[1]
            last_box_start = last_box.pos[2]
            last_box_width = last_box.width
        end
        if (last_box_shelf < aisle.num_shelves) && box.width <= aisle.size
            return true # At least one shelf left that can fit the box.
        else
            new_end = last_box_start + last_box_width + distance + box.width
            return new_end <= aisle.size # Last shelf, and box doesn't spill over
        end
    end

    function place_box!(aisle::Aisle, box::Box, distance::Float64)
        # Places a box on the aisle, wrapping the remaining distance onto the next shelf
        # if there is no room left. Assumes that there is room left in the aisle as a whole.
        if length(aisle.boxes) < 1
            last_box_shelf = 1
            last_box_start = 0
            last_box_width = 0
        else
            last_box = last(aisle.boxes)
            last_box_shelf = last_box.pos[1]
            last_box_start = last_box.pos[2]
            last_box_width = last_box.width
        end
        new_shelf = last_box_shelf
        new_start = last_box_start + last_box_width + distance
        new_end = new_start + box.width
        if new_end >= aisle.size
            new_shelf = new_shelf + 1
            if new_start <= aisle.size
                new_start = 0
            else
                new_start = new_start - aisle.size
            end
        end
        box.pos = (new_shelf, new_start)
        push!(aisle.boxes, box)
    end 
    

    function deserialize_aisle(aisle_dict)
        id = aisle_dict["id"]
        size = aisle_dict["size"]
        num_shelves = aisle_dict["num_shelves"]
        boxes = [deserialize_box(b) for b in aisle_dict["boxes"]]
        return Aisle(id, size, num_shelves, boxes)
    end 

    struct World
        aisles::Vector{Aisle}
    end 
    function deserialize_world(world_dict)
        return World([deserialize_aisle(a) for a in world_dict["aisles"]])
    end 

    struct Situation
        world::World
        location::Int
        instruction::String
        target::Int
        meaning
    end 
    function deserialize_situation(situation_dict)
        world = deserialize_world(situation_dict["world"])
        location = situation_dict["location"]
        instruction = situation_dict["instruction"]
        target = situation_dict["target"]
        meaning = situation_dict["meaning"]
        return Situation(world, location, instruction, target, meaning)
    end 

end

function load_situation(situation_file)
    curr_dir = dirname(@__DIR__)
    dataset_dir = "boxworld/data/demo_dataset"
    full_path = join([curr_dir, dataset_dir, situation_file], "/")
    open(full_path, "r") do f
        return JSON.parse(f)
    end 
    
end 

load_situation (generic function with 1 method)

Next, we'll actually define a generative model for the stockroom itself. A stockroom is a set of aisles, populated by boxes. One reasonable assumption we could make is that stockrooms there are actually 1 or more categories of boxes, and stockrooms are 
populated by placing boxes onto shelves as they are generated until no more fit.

We'll define this model below, and some helper utility distributions.

In [259]:
PARAMS = (
    num_aisles = 10,
    distance_scale = 1,
    box_scale = 0.5
)

# A distribution that is guaranteed to be 1 or higher.
@dist poisson_plus_one(rate) = poisson(rate) + 1;


@gen function stockroom_model(aisle_size::Float64, num_shelves::Float64, params)
    # Generate a categorical distribution over 1 or more box_types, each of which defines
    # it's own distribution over box heights and widths.
    num_box_types ~ poisson_plus_one(1) # Generate number of box types -- at least 1
    box_fractions ~ dirichlet([1.0 for i=1:num_box_types])
    
    shelf_height = aisle_size / num_shelves
    box_types = [
        (
         {(:box_mean_width, i)} ~ gamma(shelf_height * 0.5, shelf_height * 0.25),
         {(:box_mean_height, i)} ~ gamma(shelf_height * 0.5, shelf_height * 0.25),
        )
         for i = 1:num_box_types
    ]
    @dist choose_box_type() = box_types[categorical(box_fractions)]
    
#     num_aisles ~ gamma(params.num_aisles, 1)
    num_aisles = 1
    distance_shape ~ gamma(aisle_size / num_shelves, 1)
    distance_scale ~ gamma(params.distance_scale, 1)
    
    # For now, we;ll simply start at the start of each aisle and place boxes until no more fit.
    aisles = [Main.Stockroom.Aisle(id, aisle_size, num_shelves) for id=0:num_aisles] # N.B. 1-indexing
    curr_aisle = 1
    box_id = 1
    while curr_aisle <= length(aisles)
        box_type = {(:box_type, box_id)} ~ choose_box_type()
        box_width = {(:box_width, box_id)} ~ gamma(box_type[1], params.box_scale)
        box_height = {(:box_height, box_id)} ~ gamma(box_type[2], params.box_scale)
        box = Main.Stockroom.init_clamped_box(box_id, (0,0), box_width, box_height, aisles[curr_aisle])
        distance_from_last_box = {(:distance_from_last_box, box_id)} ~ gamma(distance_shape, distance_scale)
        if Main.Stockroom.room_left(aisles[curr_aisle], box, distance_from_last_box)
            Main.Stockroom.place_box!(aisles[curr_aisle], box, distance_from_last_box)
        else 
            if curr_aisle == length(aisles)
                break
            else
                curr_aisle += 1
                Main.Stockroom.place_box!(aisles[curr_aisle], box, distance_from_last_box)
            end 
        end 
        box_id += 1
    end
    return aisles
end 

DynamicDSLFunction{Any}(Dict{Symbol,Any}(), Dict{Symbol,Any}(), Type[Float64, Float64, Any], false, Union{Nothing, Some{Any}}[nothing, nothing, nothing], ##stockroom_model#453, Bool[0, 0, 0], false)

We'll define some visualization code to capture what these sampled stockrooms look like. This parallels the visualization viewer in boxworld/visualize.

In [260]:
aisle_size = 12
num_shelves = 4
tr = simulate(stockroom_model, (aisle_size, num_shelves, PARAMS))
get_choices(tr)

│
├── (:box_height, 12) : 0.6475449316195504
│
├── (:box_type, 12) : (0.4689672600492348, 1.5620822967321861)
│
├── (:box_height, 28) : 0.0591844491802857
│
├── (:box_width, 9) : 0.0017858025671675902
│
├── (:box_width, 45) : 0.004791022077658969
│
├── (:box_height, 34) : 0.9537283321388744
│
├── :num_box_types : 1
│
├── (:box_type, 16) : (0.4689672600492348, 1.5620822967321861)
│
├── (:box_width, 3) : 0.014840006580027022
│
├── (:box_type, 39) : (0.4689672600492348, 1.5620822967321861)
│
├── (:box_type, 18) : (0.4689672600492348, 1.5620822967321861)
│
├── (:box_type, 5) : (0.4689672600492348, 1.5620822967321861)
│
├── (:box_height, 16) : 0.13455604997260073
│
├── (:distance_from_last_box, 1) : 0.9858017391951467
│
├── (:box_height, 23) : 0.9571791335491364
│
├── (:distance_from_last_box, 31) : 1.8389483550556718
│
├── (:box_type, 40) : (0.4689672600492348, 1.5620822967321861)
│
├── (:box_height, 19) : 2.125698989591307
│
├── (:box_type, 32) : (0.4689672600492348, 1.5620822967321861)
│